In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Brownian Bridge Simulation and Metric Estimation on Landmark Manifolds
arXiv:1705.10943 [cs.CV] https://arxiv.org/abs/1705.10943

Stefan Sommer, Line Kuhnel, Alexis Arnaudon, and Sarang Joshi

In [ ]:
%cd ..
from src.manifolds.landmarks import *
M = landmarks(2)
print(M)

from src.plotting import *
%matplotlib inline
plt.rcParams['figure.figsize'] = 13, 10
colormap = plt.get_cmap('winter')

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# initialize
M.N.set_value(8)
M.k_alpha.set_value(.1)
M.k_sigma.set_value(.5*np.diag((1.,1.)))
n_steps.set_value(500)

In [ ]:
# setup 
q = np.vstack((np.linspace(-.5,.5,M.N.eval()),np.zeros(M.N.eval()))).T.flatten()
v = np.vstack((np.zeros(M.N.eval()),np.ones(M.N.eval()))).T.flatten()
p = M.flatf(q,v)
print("q = ", q)
print("p = ", p)

In [ ]:
# Hamiltonian dynamics
print(M.Hf(q,p))
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# geodesic
qs = M.Exp_Hamiltoniantf(q,v).T
M.plot()
M.plotx(qs,v)
plt.show()
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf(q,p) for (q,p) in zip(qs,ps)]))

In [ ]:
# Brownian motion
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# parameters
q0 = M.element()
thetas = (q0,M.k_alpha,M.k_sigma) # parameters

# guide function
Cholesky = T.slinalg.Cholesky()
phi = lambda q,v: T.tensordot(T.nlinalg.MatrixInverse()(Cholesky(M.gsharp(q))),-(q-v).flatten(),(1,0))

v = M.element()
phif = theano.function([q0,v],phi(q0,v))
(Brownian_coords_guided,Brownian_coords_guidedf) = get_guided_likelihood(M,M.sde_Brownian_coords,phi,lambda q: Cholesky(M.gsharp(q)),q0)

v = np.stack((np.linspace(-.5,.5,M.N.eval()),np.ones(M.N.eval()))).T.flatten()
(ts,qs,log_likelihood,log_varphi) = Brownian_coords_guidedf(q,v,dWsf(M.dim.eval()))[:4]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

# Visualize bridge
M.plot()
M.plotx(np.vstack((q,qs)),curve=True)
M.plotx(v,color='k',curve=True)
# plt.savefig('bridge.pdf')

In [ ]:
## Set up for inference example

# initialize
M.N.set_value(10)
M.k_alpha.set_value(.01)
n_steps.set_value(20)

# setup 
x = M.ellipse([0.,0.],[1.,.5])
q = x.flatten()
print("q = ", q)

avg_landmark_dist = np.mean(np.linalg.norm(x[:-1]-x[1:],axis=1))
M.k_sigma.set_value(avg_landmark_dist*np.diag((1.,1.)))
print("k_alpha: ", M.k_alpha.eval(), ", k_sigma: ", M.k_sigma.eval().flatten())
k_alpha_init = M.k_alpha.eval()
k_sigma_init = M.k_sigma.eval()

M.plotx(q,curve=True,color='k')
v = np.vstack((np.zeros(M.N.eval()),np.ones(M.N.eval()))).T
p = M.flatf(q,v.flatten())
M.plotx(M.Exp_Hamiltonianf(q,p),curve=True)

In [ ]:
# sample for Brownian motion transition distribution
thetas_true = [q]+[theta.eval() for theta in thetas[1:]]
N_samples = 64
obss = np.zeros((N_samples,)+q.shape)
qss = np.zeros((N_samples,n_steps.eval(),)+q.shape)
# srng.seed(422)
for i in range(N_samples):
    (ts,qs) = M.Brownian_coordsf(q,dWsf(M.dim.eval()))
    qss[i] = qs
    obss[i] = qs[-1]

# plot samples
plot_samples = 15
colors=[colormap(k) for k in np.linspace(0, 1, plot_samples)]
M.plot()
for i in range(plot_samples):
    M.plotx(obss[i],curve=True,color=colors[i])
# plt.savefig('samples.pdf')
plt.show()

# plot samples with paths
plot_samples = 5
colors=[colormap(k) for k in np.linspace(0, 1, plot_samples)]
M.plot()
M.plotx(q,color='k',curve=True)
for i in range(plot_samples):
    M.plotx(qss[i],color=colors[i],curve=True)
# plt.savefig('samples_paths.pdf')

In [ ]:
options = {}
options['samples_per_obs'] = 1
options['epochs'] = 80
options['learning_rate'] = .5e-3#1.5e-3
options['varphi_update_rate'] = 1.
options['initial'] = [np.zeros(M.dim.eval()),
                      np.array(.12),.3*np.diag((1.,1.))]

In [ ]:
# produce bridge plot for paper
def lbridge_sampling(thetas,*args,**kwargs):
    M.k_alpha.set_value(thetas[1])
    M.k_sigma.set_value(thetas[2])
    return partial(bridge_sampling,q,Brownian_coords_guidedf,lambda: dWsf(M.dim.eval()),options)(*args,**kwargs)

N_samples = 1
tmp0 = options['samples_per_obs']
options['samples_per_obs'] = 1
tmp1 = M.k_alpha.eval()
M.k_alpha.set_value(.01)
v = obss[0]
try:
    mpu.openPool()
    sol = mpu.pool.imap(partial(lbridge_sampling,options['initial']),mpu.inputArgs(v.reshape((1,)+v.shape),np.random.randint(1000,size=N_samples)))
    res = list(sol)
    bridges = mpu.getRes(res,0)
    log_varphis = mpu.getRes(res,1)
    log_likelihoods = mpu.getRes(res,2)
except:
    mpu.closePool()
    raise
else:
    mpu.closePool()

# plot
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, options['samples_per_obs'])]
M.plot()
for j in range(bridges.shape[1]):
    gs = np.vstack((q.flatten(),bridges[0,j]))
    M.plotx(gs,linewidth=.6,color=colors[j],curve=True)
M.plotx(v,color='b',curve=True)        
M.plotx(q,color='k',curve=True)
# plt.savefig('bridges.pdf')

options['samples_per_obs'] = tmp0
M.k_alpha.set_value(tmp1)
N_samples = obss.shape[0]

In [ ]:
# transition density etc.
v = M.element()
log_p_Tf = theano.function([q0,v],log_p_T(q0,v,dWs(M.dim),Brownian_coords_guided,phi,options,sde=M.sde_Brownian_coords))
dlog_p_Tf = theano.function([q0,v],dlog_p_T(thetas,q0,v,dWs(M.dim),Brownian_coords_guided,phi,options,sde=M.sde_Brownian_coords))
p_Tf = theano.function([q0,v],T.exp(log_p_T(q0,v,dWs(M.dim),Brownian_coords_guided,phi,options,sde=M.sde_Brownian_coords)))

v = M.Exp_Hamiltonianf(q,p)
print(log_p_Tf(q,v))
print(p_Tf(q,v))
print(dlog_p_Tf(q,v))

In [ ]:
%%time
from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (v,seed) = pars
    if seed:
        srng.seed(seed)
    q = thetas[0]
    M.k_alpha.set_value(thetas[1])
    M.k_sigma.set_value(thetas[2])
    return dlog_p_Tf(q,v)

def update_thetas(thetas, dthetas):
    q = thetas[0]
    k_alpha = thetas[1]
    k_sigma = thetas[2]
    
    q += options['learning_rate']*np.dot(M.gsharpf(q),dthetas[0]) # use Riemannian g-gradient
    k_alpha += options['learning_rate']/M.dim.eval()*dthetas[1]
    k_sigma += options['learning_rate']*dthetas[2]
    
    return (q,k_alpha,k_sigma)


# initial values
options['initial'][0] = np.mean(obss,axis=0)

# run MLE
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss,llog_p_T,update_thetas,options)

## plot
plt.plot(range(options['epochs']),log_likelihoods)
# plt.savefig('likelihood.pdf')
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
# plt.savefig('q0s.pdf')
plt.show()
plt.plot(range(options['epochs']),thetass[1],color='b')
plt.hlines(thetas_true[1],plt.xlim()[0],plt.xlim()[1],color='r')
# plt.savefig('k_alpha.pdf')
plt.show()
plt.plot(range(options['epochs']),thetass[2].reshape((thetass[2].shape[0],-1)),color='b')
plt.hlines(thetas_true[2].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.ylabel(r'$\sigma$', fontsize=30)
# plt.savefig('k_sigma.pdf')
plt.show()
M.plotx(thetas_true[0].flatten(),color='k',curve=True)
M.plotx(thetas[0],color='b',curve=True)
# plt.savefig('est_q0.pdf')
plt.show()
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(range(options['epochs']),log_likelihoods,'g--')
ax1.set_ylabel(r'$\mathcal{L}_\theta$', fontsize=30)
ax2.plot(range(options['epochs']),thetass[1],color='b')
ax2.hlines(options['initial'][1],plt.xlim()[0],plt.xlim()[1],color='r')
ax2.set_ylabel(r'$\alpha$', fontsize=30)
# plt.savefig('likelihood-k_alpha.pdf')
plt.show()
None

In [ ]:
# sample with estimated parameters
obss_new = np.zeros((N_samples,)+q.shape)
for i in range(N_samples):
    (ts,qs) = M.Brownian_coordsf(q,dWsf(M.dim.eval()))
    qss[i] = qs
    obss_new[i] = qs[-1]

In [ ]:
# plot
def estimate_qq(data_q):
    data_mean= data_q.sum(0)/data_q.shape[0]
    data= data_q - data_mean
    
    return [data_mean,(data[:,:,:,np.newaxis,np.newaxis]*data[:,np.newaxis,np.newaxis,:,:]).sum(0)/data.shape[0]]
qq = estimate_qq(obss.reshape((-1,M.N.eval(),M.m.eval())))
qq_new = estimate_qq(obss_new.reshape((-1,M.N.eval(),M.m.eval())))

#plot density distribution of landmarks
def plot_distribution(xss):
    xTx=[]
    xTy=[]
    for i in range(xss.shape[0]):
        for j in range(0,M.N.eval()):
            xTx.append(xss[i,j,0])
            xTy.append(xss[i,j,1])
    hist,histy,histx= np.histogram2d(xTy,xTx,bins=25)
    extent = [histx[0],histx[-1],histy[0],histy[-1]]

    
    #plt.contour(hist/np.max(hist),extent=extent,levels=[0.05,0.2,0.4,0.6],zorder=10)
    plt.imshow(hist/np.max(hist),extent=extent,interpolation='bicubic',origin='lower',cmap='Greys')#,levels=[0.05,0.2,0.4,0.6],zorder=10)
    #plt.colorbar()

# plot variance
def plot_final_ellipses(q,QQ,coeff=1.,c='m',ls='-',lw=1):
    # plot sigma as ellipses 
    from matplotlib.patches import Ellipse
    from numpy import linalg as LA
    ax= plt.gca()
    for i in range(M.N.eval()):
        qq_eig,qq_vec = LA.eig(QQ[i,:,i,:])
        qq_eig = np.sqrt(qq_eig)
        theta = np.degrees(np.arctan(qq_vec[1,0]/qq_vec[0,0]))

        ell= Ellipse(xy=q[i] ,width=coeff*qq_eig[0],height= coeff*qq_eig[1],angle=theta,ls=ls,lw=lw)
        ax.add_artist(ell)
        ell.set_alpha(1.)
        ell.set_facecolor('None')
        ell.set_edgecolor(c)
  
M.plot()
M.plotx(x.flatten(),color='k',curve=True)
M.plotx(q,color='b',curve=True)
plot_final_ellipses(qq[0],qq[1],coeff=3.,c='k',ls='-',lw=2)
plot_final_ellipses(q.reshape((-1,M.m.eval())),qq_new[1],coeff=3.,c='b',ls='--',lw=2)
plot_distribution(obss_new.reshape((-1,M.N.eval(),M.m.eval())))
# plt.savefig('ellipse_inf.pdf')